In [1]:
# pip install jupyterthemes

In [2]:
# !jt -l

In [3]:
# !jt -t monokai

In [4]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text 

# cls sentance sep cls means begining sep seperats two differnt sentance

In [5]:
import pandas as pd 
df = pd.read_csv('dataset/spamdetection.csv',sep='\t',names=['label','message']).dropna()
df.head(10)

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


In [6]:
v = df['label'].value_counts()
ham = df.loc[df['label'] == 'ham'][0:254]
spam = df.loc[df['label']== 'spam'][0:254]
df = pd.concat([ham,spam]).sample(frac=1)


In [7]:
x = list(df['message'])
y = list(df['label'])


In [8]:
y = list(pd.get_dummies(y,drop_first=True) ['spam']) #take spam column then convert to list


In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.20,random_state=0)

In [10]:
encoder_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4'
preprocess_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'
bert_preprocess = hub.KerasLayer(preprocess_url)
bert_encoder = hub.KerasLayer(encoder_url)

In [11]:
#bert layers
text_input = tf.keras.layers.Input(shape=(),dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
output = bert_encoder(preprocessed_text)


SyntaxError: invalid syntax (Temp/ipykernel_9024/3232608611.py, line 2)